In [5]:
!pip install opencv-python

In [3]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#     transforms.Grayscale(num_output_channels=1),
])

dataset = MNIST('./data', download=True, transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder()#.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)


In [4]:
# for epoch in range(num_epochs):
#     for data in dataloader:
#         img, _ = data
#         #print(img.shape)
#         img = img.view(img.size(0), -1)
#         img = Variable(img)#.cuda()
#         output = model(img)
#         loss = criterion(output, img)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#     print('epoch [{}/{}], loss:{:.4f}'
#           .format(epoch + 1, num_epochs, loss.item()))
#     if epoch % 10 == 0:
#         pic = to_img(output.cpu().data)
#         save_image(pic, './mlp_img/image_{}.png'.format(epoch))

        
# torch.save(model.state_dict(), './sim_autoencoder.pth')

In [5]:
# torch.save(model, './sim_autoencoder.pth')
model.load_state_dict(torch.load('./sim_autoencoder.pth', map_location='cpu'))

<All keys matched successfully>

In [6]:
import numpy as np
import matplotlib.pyplot as plt


#model = torch.load('./sim_autoencoder.pth', map_location='cpu')
img, _ = dataset[0]
#print(img)
my_img = model(img.reshape(28*28))
my_img = my_img.reshape(1,28,28)
#my_img.shape
results = transforms.ToPILImage(mode='L')(my_img)
results.show()
#output[0]
#image = output.cpu().numpy()[0]
#image = np.transpose(image, (1,2,0))
#plt.matshow(image)
#plt.show()

In [7]:
img, _ = dataset[0]
results = transforms.ToPILImage(mode='L')(img.reshape(1,28,28)) 
results.show()

img2 = model(img.reshape(28*28))
results2 = transforms.ToPILImage(mode='L')(img2.reshape(1,28,28)) 
results2.show()



In [15]:
#show middle
img, _ = dataset[0]
img2, _ = dataset[1]
enc1 = model.encoder(img.reshape(28*28))
enc2 = model.encoder(img2.reshape(28*28))

In [19]:
result = (enc1+enc2)/2

In [58]:
def toPILimg(img):
    return transforms.ToPILImage(mode='L')(img.reshape(1,28,28))

def encode(x):
    form_img = x.reshape(28*28)
    return model.encoder(form_img)

def decode(enc):
    dec_img = model.decoder(enc)
    return toPILimg(dec_img)

In [59]:
x = encode_img(img2)
y = decode_enc(x)
y.show()

In [61]:
img5_2, _ = dataset[11]
img5_1, _ = dataset[0]
toPILimg(img5_1).show()
toPILimg(img5_2).show()
decode((encode(img5_1) + encode(img5_2))/2).show()